# Povezivanje sa drive-om i git-om



In [2]:
#spajam ga sa mojim drajvom gde su svi fajlovi
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#tačna lokacija
%cd "drive/My Drive/neural_style_transfer_audio"

/content/drive/My Drive/neural_style_transfer_audio


In [ ]:
!git clone https://github.com/Emilija2000/PSIML6_Voice_style_transfer.git

In [4]:
%cd PSIML6_Voice_style_transfer/

/content/drive/My Drive/neural_style_transfer_audio/PSIML6_Voice_style_transfer


In [6]:
!git pull

Already up to date.


#Import svih potrebnih stvari

In [5]:
!pip install pydub
!pip install torchaudio
!pip install scipy

     |████████████████████████████████| 6.7MB 2.6MB/s 


In [ ]:
#ubacivanje nasih fajlova kao biblioteka
import dataloader

In [ ]:
dataset = dataloader.AccentDataset('data/recordings/recordings','data/speakers_all.csv')
print (dataset.__len__())
vect,label = dataset.__getitem__(1)

#Prekopiran ceo kod - isprobavanje

In [111]:
from collections import defaultdict
import copy
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
from pydub import AudioSegment
import scipy
import scipy.signal
from scipy.io import wavfile
import time
import torch
import torchvision
import librosa                    
import librosa.display


AudioSegment.converter = "ffmpeg"
allowed_genres = ["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]

def readAudio(mp3_path):
    '''
    Read audio file on location mp3_path using librosa and creates spectrogram
    '''
    y, sr = librosa.load(mp3_path)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    melSpec_dB = librosa.power_to_db(melSpec, ref=np.max)
    #librosa.display.specshow(melSpec_dB)
    return melSpec_dB, sr

def writeAudio(spectrogram, sample_rate, output_file_name):
    '''
    For given scipy spectrogram and sample_rate, write .wav audiofile
    with the name output_file_name + '.wav'
    '''
    audio_signal = librosa.core.spectrum.griffinlim(spectrogram)
    #print(audio_signal, audio_signal.shape)
    # write output
    file_name = output_file_name + '.wav'
    scipy.io.wavfile.write(file_name, sample_rate, np.array(audio_signal, dtype=np.int16))

def readSpectrogram(png_path):
    '''
    Reads png file and save spectrogram as a matrix
    '''
    pass

class MusicDataset(torch.utils.data.Dataset):
    def __init__(self, path, csv_path):
        self.data_names = []    #mp3 file location and data label
        self.data_names_onehot = [] #mp3 file location and onehot encoded data label
        self.classes = [] #all classes (possible labels)
        
        with open(csv_path) as csvfile:
            #start cvs reader
            csv_reader = csv.reader(csvfile,delimiter=',')
            next(csv_reader) #skip first row
            for row in csv_reader:
                if (row[59] in allowed_genres): 
                    mp3_path = os.path.join(path,row[59],row[0])
                    self.data_names.append((mp3_path, row[59])) 
                    if (row[59] not in self.classes):
                        self.classes.append(row[59])

            #create onehot encoding for data labels
            for data in self.data_names:
                label = torch.zeros(len(self.classes)) 
                label[self.classes.index(data[1])] = 1
                self.data_names_onehot.append((data[0],label))

    def __getitem__(self,index):
        #read audiofile
        spect, sr = readAudio(self.data_names[index][0])

        #get onehot encoded label
        label_onehot = self.data_names_onehot[index][1]
        label_onehot = torch.Tensor(label_onehot)
        #label_onehot = label.type(torch.long)
        #label_onehot = label.reshape(len(label),1,1)

        #int encoding for classes
        label = torch.argmax(label_onehot).item()

        #reshape spectrogram to represent 1 channel
        spect = spect[:,:1280]
        spect = torch.Tensor(spect).reshape(1,spect.shape[0],spect.shape[1])

        return spect, label

    def __len__(self):
        return len(self.data_names)
    

In [112]:
#isprobavanje
dataset = MusicDataset('Data/genres_original','Data/features_30_sec.csv')
print('Velicina dataseta je: ',dataset.__len__())
dat,label = dataset.__getitem__(1)
print(dat.size())


Velicina dataseta je:  1000
torch.Size([1, 128, 1280])


#Model klasifikatora


In [113]:
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, output_num):
        super(Classifier,self).__init__()
        modules = []

        modules.append(nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(8))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(16))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(32))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(64))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(128))
        modules.append(nn.ReLU(inplace = True))


        modules.append(nn.AdaptiveAvgPool2d(output_size = 1))


        self.fc=nn.Linear(in_features=128,out_features=output_num,bias=True)

        self.sequence = nn.Sequential(*modules)


    def forward(self,x):
        y = self.sequence(x)
        y=torch.flatten(y,start_dim=1)
        y = self.fc(y)
        return y


In [114]:
#create classifier and move to device
classifier = Classifier(len(dataset.classes))
device = torch.device('cuda')
classifier = classifier.cuda()

#Trening klasifikatora

In [115]:
learning_rate = 0.001
optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)
loss_func = nn.CrossEntropyLoss()

In [116]:
# make train and valid splits
import random
random.seed(0)  # rng seed, set to 0 for reproducibility

dataset_indices = list(range(len(dataset)))
random.shuffle(dataset_indices)  # shuffle the indices before splitting (use random.shuffle)

#split datasets
train_split_indices = dataset_indices[:int(len(dataset_indices)*0.6)]  # get the training split indices
valid_split_indices = dataset_indices[int(len(dataset_indices)*0.6):int(len(dataset_indices)*0.8)]  # get the validation split indices 
test_split_indices = dataset_indices[int(len(dataset_indices)*0.8):]

train_subset_sampler = torch.utils.data.SubsetRandomSampler(train_split_indices)
valid_subset_sampler = torch.utils.data.SubsetRandomSampler(valid_split_indices)
test_subset_sampler = torch.utils.data.SubsetRandomSampler(test_split_indices)


In [117]:
batch_size = 10
num_workers = 0

train_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=train_subset_sampler, num_workers=num_workers, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=valid_subset_sampler, num_workers=num_workers, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=test_split_indices, num_workers=num_workers, drop_last=True)

In [120]:
num_epochs = 10

for epoch in range(num_epochs):
    classifier.train()
    
    for i, (spect,label) in enumerate(train_dataloader):
        spect = spect.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = classifier(spect)
        #print("output=",output," label=",label)
        loss = loss_func(output, label)

        loss.backward()
        optimizer.step()

        if (i+1) % (1) == 0:
            print(f'epoch: {epoch}   iter: {i+1}   batch_loss: {loss}')
            #print('output: ', output,' label: ',label)

    classifier.eval()
    
    correct = 0
    total = 0

    with torch.no_grad():
        for (spect,label) in valid_dataloader:
            spect = spect.to(device)
            label = label.to(device)

            output = classifier(spect)

            y_pred = torch.log_softmax(output,-1)  # convert logits (model outputs) to class probabilities

            , predicted = torch.max(y_pred, 1)  # find the most probable class (use torch.max)

            total+= y.size(0)
            correct+= (int(predicted == y)).sum()


    validation_accuracy = 100 * float(correct)/total
    print(f'epoch: {epoch}   validation accuracy: {validation_accuracy}%' )


        
        

epoch: 0   iter: 1   batch_loss: 1.5329313278198242
epoch: 0   iter: 2   batch_loss: 1.577140212059021
epoch: 0   iter: 3   batch_loss: 1.643634557723999
epoch: 0   iter: 4   batch_loss: 1.9305698871612549
epoch: 0   iter: 5   batch_loss: 1.9317677021026611
epoch: 0   iter: 6   batch_loss: 1.9223048686981201
epoch: 0   iter: 7   batch_loss: 1.8988357782363892
epoch: 0   iter: 8   batch_loss: 1.50393807888031
epoch: 0   iter: 9   batch_loss: 1.8164136409759521
epoch: 0   iter: 10   batch_loss: 1.792891502380371
epoch: 0   iter: 11   batch_loss: 1.495210886001587
epoch: 0   iter: 12   batch_loss: 1.6918067932128906
epoch: 0   iter: 13   batch_loss: 1.874091386795044
epoch: 0   iter: 14   batch_loss: 1.8176500797271729
epoch: 0   iter: 15   batch_loss: 1.9528436660766602
epoch: 0   iter: 16   batch_loss: 1.7618160247802734
epoch: 0   iter: 17   batch_loss: 1.636839509010315
epoch: 0   iter: 18   batch_loss: 1.4146817922592163
epoch: 0   iter: 19   batch_loss: 1.626970887184143


KeyboardInterrupt: ignored